# Travel Planner

A simple implementation with routing

In [40]:
import os
from dotenv import load_dotenv
from typing import Any, Dict, List, Optional, Literal, Set, TypedDict
from pydantic import BaseModel, Field, ValidationError
from IPython.display import display, Image, Markdown
import json
import datetime
# LangGraph StateGraph import (assume installed)
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

# LangChain imports (LLM & Chains)
from langchain_openai import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Tavily search
from langchain_tavily import TavilySearch

In [3]:
load_dotenv()

True

In [4]:
# Setup LLM clients using env keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Set it in .env or env vars before running.")

llm = ChatOpenAI(model="gpt-4", temperature=0.0)

In [5]:
# Setup Tavily client (assume TAVILY_API_KEY present)
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise RuntimeError("TAVILY_API_KEY not set in environment. Set it in .env or env vars before running.")

# instantiate tool
tavily_tool = TavilySearch(max_results = 5)

In [6]:
class Constraints(BaseModel):
    origin: Optional[str] = None
    city: Optional[str] = None
    date: Optional[str] = None
    return_date: Optional[str] = None
    days: Optional[int] = None
    budget_per_person: Optional[float] = None
    interests: Optional[str] = None
    transport_constraints: Optional[Dict[str, Any]] = None
    hotel_constraints: Optional[Dict[str, Any]] = None

class TransportOption(BaseModel):
    mode: str
    depart: str
    arrive: str
    duration_min: Optional[str]
    price_per_person: Optional[str]
    notes: Optional[str] = None

class HotelOption(BaseModel):
    name: str
    rating: Optional[float]
    price_per_night: float
    distance_from_center_km: Optional[float]
    amenities: List[str] = Field(default_factory=list)
    notes: Optional[str] = None

class ItineraryItem(BaseModel):
    day: int
    morning: str
    afternoon: str
    evening: str
    notes: Optional[str] = None

class Intention(BaseModel):
    type: Literal["travel_plan","hotel_only","transport_only","unrelated"]

class PlannerState(TypedDict):
    goal: str
    intent: Optional[str]
    constraints: Optional[Constraints] = None
    transport_options: List[TransportOption] = Field(default_factory=list)
    hotel_options: List[HotelOption] = Field(default_factory=list)
    itinerary: List[ItineraryItem] = Field(default_factory=list)
    # subtasks: List[Subtask] = Field(default_factory=list)
    # policy: Dict[str, Any] = Field(default_factory=lambda: {"max_retries": 2, "worker_timeout": 12.0})
    final_plan: Optional[str] = None

In [7]:
# Supervisor LLM with intentation output
supervisor_llm = llm.with_structured_output(Intention, method="function_calling")

In [8]:
# Supervisor prompt to classify user intent
supervisor_prompt = PromptTemplate(
    input_variables=["goal"],
    template=(
        "You are an expert supervisor of a Travel Planner.\n"
        "Classify the user's travel planning intent based on the goal below.\n"
        "Possible intents are:\n"
        "1. travel_plan - planning a full trip\n"
        "2. hotel_only - only looking for hotel options\n"
        "3. transport_only - only looking for transport options\n"
        "4. unrelated - not related to travel planning or transport booking or hotel booking\n\n"
        "Goal: {goal}\n\n"
        "Respond with ONLY the intent type as one of these exact values: travel_plan, hotel_only, transport_only, unrelated.\n"
        "Do NOT include any explanation, markdown, or extra text. Output only the value."
    )
)

# Supervisor chain
supervisor_chain = supervisor_prompt | supervisor_llm

# Planner promt and chain
constraints_parser = PydanticOutputParser(pydantic_object=Constraints)
format_instructions = constraints_parser.get_format_instructions()

planner_prompt_template = f"""
You are an assistant that extracts travel constraints from a user query.
Return ONLY valid JSON matching the Constraints model below as a single JSON object.
Include all keys in the output, even if the value is null.
If the user query contains any specific requirements for transport (such as preferred mode, timing, class, etc.), include them as key-value pairs inside the "transport_constraints" object. If there are specific hotel requirements (such as star rating, amenities, location, etc.), include them as key-value pairs inside the "hotel_constraints" object. If there are no such requirements, set these fields to null.
Do NOT include any explanation, markdown, or code block formatting (no triple backticks).
Do NOT include trailing commas in the JSON.

Constraints model:
  "origin": string or null,
  "city": string or null,
  "date": string (YYYY-MM-DD) or null,
  "return_date": string (YYYY-MM-DD) or null,
  "days": integer or null,
  "budget_per_person": float or null,
  "interests": string or null,
  "transport_constraints": object or null,
  "hotel_constraints": object or null

User query:
{{text}}

Respond ONLY with valid JSON matching the model above.
"""

planner_prompt = PromptTemplate(
    input_variables=["text"],
    template=planner_prompt_template
)
planner_chain = planner_prompt | llm

In [9]:
# Define supervisor node function
def supervisor_node(state: PlannerState) -> PlannerState:
    goal = state.get("goal")
    if not goal:
        raise ValueError("Goal is required in the state.")

    result = supervisor_chain.invoke({"goal": goal})
    print(f"Supervisor raw output: {result.type}")
    try:
        
        return {
            "goal": goal,
            "intent": result.type,
            "final_plan": None
        }
    except ValidationError as e:
        print(f"Validation error parsing intention: {e}")
        return {
            "goal": goal,
            "intent": "unrelated",
            "final_plan": None
        }

In [10]:
# Define supervisor gate
def supervisor_gate(state: PlannerState) -> Literal["plan", "unrelated"]:
    if state.get("intent") in ["travel_plan", "hotel_only", "transport_only"]:
        return "plan"
    return "unrelated"

In [61]:
def planner_node(state: PlannerState):
    goal = state.get("goal")
    if not goal:
        raise ValueError("Goal is required in the state.")

    try:
        constraints = planner_chain.invoke({"text": goal})
        print(constraints.content)
        constraints_model = constraints_parser.parse(constraints.content)
    except Exception as e:
        print("Extractor parse error:", e)
        constraints_model = Constraints()

    # Transport intent: require origin and city
    if state.get("intent") == "transport_only":
        missing = []
        if not constraints_model.origin:
            missing.append("origin")
        if not constraints_model.city:
            missing.append("destination city")
        if missing:
            message = f"To find transport options, please provide the following missing information: {', '.join(missing)}."
            return Command(goto=END, update={"final_plan": {"message": message}})
        return Command(goto="transport", update={"constraints": constraints_model})

    # Hotel intent: require city
    elif state.get("intent") == "hotel_only":
        if not constraints_model.city:
            message = "To find hotel options, please provide the destination city."
            return Command(goto=END, update={"final_plan": {"message": message}})
        return Command(goto="hotel", update={"constraints": constraints_model})

    else:
        missing = []
        if not constraints_model.origin:
            missing.append("origin")
        if not constraints_model.city:
            missing.append("destination city")
        if missing:
            message = f"To find transport options, please provide the following missing information: {', '.join(missing)}."
            return Command(goto=END, update={"final_plan": {"message": message}})
        if not constraints_model.city:
            message = "To find hotel options, please provide the destination city."
            return Command(goto=END, update={"final_plan": {"message": message}})
        return Command(goto=["transport", "hotel", "itinerary"], update={"constraints": constraints_model})


In [12]:
# Define end unrelated node
def end_unrelated_node(state: PlannerState) -> PlannerState:
    return {
        "goal": state["goal"],
        "intent": state["intent"],
        "final_plan": {"message": "The request was unrelated to travel planning."}
    }

In [ ]:
# Define the prompt templates and chains for transport agent, hotel_agent, and itinerary_agent

transport_prompt_template = (
    "You are an expert travel assistant. Extract and return a JSON array of TransportOption objects (at least one) "
    "from the following transport search results. Respond ONLY with valid JSON matching the model below. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw transport search results:\n"
    "{search_results}\n\n"
    "TransportOption model:\n"
    "  \"mode\": string,\n"
    "  \"depart\": string,\n"
    "  \"arrive\": string,\n"
    "  \"duration_min\": string or null,\n"
    "  \"price_per_person\": string or null,\n"
    "  \"notes\": string or null"
)
transport_prompt = PromptTemplate(
    input_variables=["search_results"],
    template=transport_prompt_template
)
transport_chain = transport_prompt | llm

hotel_prompt_template = (
    "You are an expert travel assistant. Extract and return a JSON array of HotelOption objects (at least one) "
    "from the following hotel search results. Respond ONLY with valid JSON matching the model below. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw hotel search results:\n"
    "{search_results}\n\n"
    "HotelOption model:\n"
    "  \"name\": string,\n"
    "  \"rating\": float or null,\n"
    "  \"price_per_night\": float or 0,\n"
    "  \"distance_from_center_km\": float or null,\n"
    "  \"amenities\": array of strings,\n"
    "  \"notes\": string or null"
)
hotel_prompt = PromptTemplate(
    input_variables=["search_results"],
    template=hotel_prompt_template
)
hotel_chain = hotel_prompt | llm

itinerary_prompt_template = (
    "You are an expert travel assistant that creates a day-wise itinerary based on the following search results and user constraints. "
    "Your response must be a JSON array of ItineraryItem objects. "
    "Do not include any explanation, markdown, or extra text.\n\n"
    "Raw itinerary search results:\n"
    "{search_results}\n\n"
    "User constraints:\n"
    "{constraints}\n\n"
    "ItineraryItem model:\n"
    "  \"day\": integer,\n"
    "  \"morning\": string,\n"
    "  \"afternoon\": string,\n"
    "  \"evening\": string,\n"
    "  \"notes\": string or null (optional)\n"
    "Create a detailed day-wise itinerary that matches the constraints."
)
itinerary_prompt = PromptTemplate(
    input_variables=["search_results", "constraints"],
    template=itinerary_prompt_template
)
itinerary_chain = itinerary_prompt | llm

In [43]:
#  Define the transport node
def transport_node(state: PlannerState) -> PlannerState:
    print("In transport node")
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for transport planning.")

    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    origin = constraints_dict.get("origin") or "any city in India"
    city = constraints_dict.get("city") or "any city in India"
    date = constraints_dict.get("date") or datetime.date.today().isoformat()
    days = constraints_dict.get("days") or 1
    budget = constraints_dict.get("budget_per_person") or "any"
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Transport options from {origin} to {city} on {date} for {days} day(s), "
        f"budget {budget}, interests: {interests}. "
        "Return provider, mode, depart, arrive, duration in minutes, price per person, and notes."
    )
    print("Transport search query:", query)

    search_result = tavily_tool.invoke({"query": query})
    print("Raw transport search result:", search_result)

    llm_input = {"search_results": search_result}

    try:
        transport_options = transport_chain.invoke(llm_input)
        print("LLM transport options raw output:", transport_options)
        transport_options = getattr(transport_options, "content", transport_options)
        print("LLM transport options content:", transport_options)
        transport_options_list = json.loads(transport_options)
        transport_options = [TransportOption(**item) for item in transport_options_list]
    except Exception as e:
        print("Transport parse error:", e)
        transport_options = []

    return {
        "transport_options": transport_options
    }

In [44]:
# Define the hotel node
def hotel_node(state: PlannerState) -> PlannerState:
    print("In hotel node")
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for hotel planning.")

    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    city = constraints_dict.get("city") or "any city in India"
    # Set date to today if not present
    date = constraints_dict.get("date") or datetime.date.today().isoformat()
    days = constraints_dict.get("days") or 1
    budget = constraints_dict.get("budget_per_person") or "any"
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Hotels in {city} check-in on {date} for {days} night(s), "
        f"budget {budget}, interests: {interests}. "
        "Return hotel name, rating, price per night, distance from center, and amenities."
    )
    print("Hotel search query:", query)

    search_result = tavily_tool.invoke({"query": query})
    print("Raw search result:", search_result)

    llm_input = {"search_results": search_result}

    try:
        hotel_options = hotel_chain.invoke(llm_input)
        print("LLM hotel options raw output:", hotel_options)
        hotel_options = getattr(hotel_options, "content", hotel_options)
        print("LLM hotel options content:", hotel_options)
        hotel_options_list = json.loads(hotel_options)
        hotel_options = [HotelOption(**item) for item in hotel_options_list]
    except Exception as e:
        print("Hotel parse error:", e)
        hotel_options = []

    return {
        "hotel_options": hotel_options
    }

In [16]:
# Define itinerary node
def itinerary_node(state: PlannerState) -> PlannerState:
    constraints = state.get("constraints")
    if not constraints:
        raise ValueError("Constraints are required in the state for itinerary planning.")

    # Prepare a query for Tavily based on constraints
    constraints_dict = constraints.model_dump() if hasattr(constraints, "dict") else constraints
    city = constraints_dict.get("city") or "any city in India"
    days = constraints_dict.get("days") or 1
    interests = constraints_dict.get("interests") or "none"
    query = (
        f"Tourist attractions and activities in {city} for a {days}-day trip. "
        f"Interests: {interests}. Suggest morning, afternoon, and evening activities for each day."
    )
    print("Itinerary search query:", query)

    # Call Tavily tool to get search results
    search_result = tavily_tool.invoke({"query": query})
    print("Raw itinerary search result:", search_result)

    # Prepare input for LLM
    constraints_str = (
        json.dumps(constraints_dict, indent=2)
        if isinstance(constraints_dict, dict)
        else str(constraints_dict)
    )
    llm_input = {
        "search_results": search_result,
        "constraints": constraints_str
    }

    try:
        itinerary = itinerary_chain.invoke(llm_input)
        print(f"LLM itinerary raw output: {itinerary}")
        itinerary_content = getattr(itinerary, "content", itinerary)
        print(f"LLM itinerary content: {itinerary_content}")
        itinerary_list = json.loads(itinerary_content)
        print(f"Parsed itinerary_list: {itinerary_list}")
        if not isinstance(itinerary_list, list):
            raise ValueError("LLM did not return a JSON array.")
        itinerary = [ItineraryItem(**item) for item in itinerary_list]
    except Exception as e:
        print("Itinerary parse error:", e)
        itinerary = []

    return {
        "itinerary": itinerary
    }

In [45]:
# Define prompt templates and chains for final plan summarization# For full travel plan.
final_plan_prompt_template_travel_plan = """
You are an expert travel assistant that summarizes the final travel plan based on user constraints, transport options, hotel options, and itinerary.
Your response must be a summarized text containing the final travel plan details with the following sections:
1. Overview: A brief summary of the trip including origin, destination, dates, duration, budget, and interests.
2. Transport Options: A summary of the selected transport options including provider, mode, depart, arrive, duration, price, and notes.
3. Hotel Options: STRICTLY list and describe ONLY the hotel options provided below. DO NOT invent, generalize, or add any hotels or details not present in the hotel options list. For each hotel, include its name, rating, price, distance from center, amenities, and notes exactly as given.
4. Itinerary: A day-wise breakdown of the itinerary including morning, afternoon, evening activities and notes.
5. Estimated Total Cost: A summary of the estimated total cost for transport and hotel per person and itinerary activities.

User constraints:
{constraints}
Transport options:
{transport_options}
Hotel options:
{hotel_options}
Itinerary:
{itinerary}

Summarize the final travel plan in a clear and concise manner in a markdown format.
"""

# For transport only
final_plan_prompt_template_transport_only = """
You are an expert travel assistant. Summarize ONLY the transport options for the user's travel request, based on the user constraints and transport options below.
Do NOT include hotel or itinerary information. The output must be a summarized text containing the transport details with the following sections:
1. Overview: A brief summary of the trip including origin, destination, dates, duration, budget, and interests.
2. Transport Options: A summary of the selected transport options including mode, depart, arrive, duration, price, and notes.

User constraints:
{constraints}
Transport options:
{transport_options}

Summarize the available transport options in a clear and concise manner in a markdown format.
"""

# For hotel only
final_plan_prompt_template_hotel_only = """
You are an expert travel assistant. Summarize ONLY the hotel options for the user's travel request, based on the user constraints and hotel options below.
Do NOT include transport or itinerary information. The output must be a summarized text containing the hotel details with the following sections:
1. Overview: A brief summary of the trip including destination, dates, duration, budget, and interests.
2. Hotel Options: STRICTLY list and describe ONLY the hotel options provided below.

User constraints:
{constraints}
Hotel options:
{hotel_options}

Summarize the available hotel options in a clear and concise manner in a markdown format.
"""


In [46]:
# Define the summarizer node
def summarizer_node(state: PlannerState) -> PlannerState:
    constraints = state.get("constraints")
    transport_options = state.get("transport_options", [])
    hotel_options = state.get("hotel_options", [])
    itinerary = state.get("itinerary", [])
    intent = state.get("intent")

    if not constraints:
        raise ValueError("Constraints are required in the state for final plan summarization.")

    # Select prompt based on intent
    if intent == "transport_only":
        if not transport_options:
            return {
                "goal": state["goal"],
                "intent": intent,
                "constraints": constraints,
                "transport_options": transport_options,
                "hotel_options": hotel_options,
                "itinerary": itinerary,
                "final_plan": {"message": "No transport options were found."}
            }
        prompt = PromptTemplate(
            input_variables=["constraints", "transport_options"],
            template=final_plan_prompt_template_transport_only
        )
        llm_input = {
            "constraints": constraints,
            "transport_options": transport_options,
        }
    elif intent == "hotel_only":
        if not hotel_options:
            return {
                "goal": state["goal"],
                "intent": intent,
                "constraints": constraints,
                "transport_options": transport_options,
                "hotel_options": hotel_options,
                "itinerary": itinerary,
                "final_plan": {"message": "No hotel options were found."}
            }
        prompt = PromptTemplate(
            input_variables=["constraints", "hotel_options"],
            template=final_plan_prompt_template_hotel_only
        )
        llm_input = {
            "constraints": constraints,
            "hotel_options": hotel_options,
        }
    else:  # travel_plan or fallback
        prompt = PromptTemplate(
            input_variables=["constraints", "transport_options", "hotel_options", "itinerary"],
            template=final_plan_prompt_template_travel_plan
        )
        llm_input = {
            "constraints": constraints,
            "transport_options": transport_options,
            "hotel_options": hotel_options,
            "itinerary": itinerary,
        }

    final_plan_chain = prompt | llm

    try:
        final_plan_summary = final_plan_chain.invoke(llm_input)
        print(f"Final plan summary raw output: {final_plan_summary}")
    except Exception as e:
        print("Final plan summarization parse error:", e)
        final_plan_summary = {"message": "Could not generate final plan summary."}

    return {
        "goal": state["goal"],
        "intent": intent,
        "constraints": constraints,
        "transport_options": transport_options,
        "hotel_options": hotel_options,
        "itinerary": itinerary,
        "final_plan": final_plan_summary
    }

In [62]:
# Build and complile the state graph
builder = StateGraph(PlannerState)

# Add nodes
builder.add_node("supervisor", supervisor_node)
builder.add_node("planner", planner_node)
builder.add_node("end_unrelated", end_unrelated_node)
builder.add_node("transport", transport_node)
builder.add_node("hotel", hotel_node)
builder.add_node("itinerary", itinerary_node)
builder.add_node("final_summarizer", summarizer_node)

# Add edges
builder.add_edge(START, "supervisor")
builder.add_conditional_edges("supervisor", supervisor_gate, {
    "plan": "planner",
    "unrelated": "end_unrelated"
})
# builder.add_edge("planner", END)
builder.add_edge("transport", "final_summarizer")
builder.add_edge("hotel", "final_summarizer")
builder.add_edge("itinerary", "final_summarizer")
builder.add_edge("final_summarizer", END)
builder.add_edge("end_unrelated", END)

# Compile the graph
graph = builder.compile()

In [63]:
# Test the graph with a travel planning goal
input_text = "Plan a 3-day trip to Goa for two adults,and one child from Asansol by train and we need a 3-star hotel with breakfast included"
initial_state = {
    "goal": input_text,
    "intent": None,
    "final_plan": None
}
result = graph.invoke(initial_state)
print(result)

Supervisor raw output: travel_plan
{
  "origin": "Asansol",
  "city": "Goa",
  "date": null,
  "return_date": null,
  "days": 3,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": {
    "mode": "train",
    "adults": 2,
    "children": 1
  },
  "hotel_constraints": {
    "star_rating": 3,
    "amenities": "breakfast included"
  }
}
In hotel node
Hotel search query: Hotels in Goa check-in on 2025-09-19 for 3 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.
Itinerary search query: Tourist attractions and activities in Goa for a 3-day trip. Interests: none. Suggest morning, afternoon, and evening activities for each day.
In transport node
Transport search query: Transport options from Asansol to Goa on 2025-09-19 for 3 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.
Raw itinerary search result: {'query': 'Tourist att

In [64]:
if result.get("final_plan"):
    if isinstance(result["final_plan"], dict) and "message" in result["final_plan"]:
        display(Markdown(result["final_plan"]["message"]))
    else:
        display(Markdown(result["final_plan"].content))

# Final Travel Plan

## 1. Overview
The trip is planned for a family of 2 adults and 1 child from Asansol to Goa for a duration of 3 days. The mode of transport preferred is train and the hotel preference is a 3-star rating with breakfast included. The exact dates of travel are yet to be decided.

## 2. Transport Options
The recommended way to reach Goa from Asansol is by Train. However, the quickest way to get from Asansol to Goa is a combination of train, plane, taxi, and bus, which takes approximately 1035 minutes and costs $301 per person. The cheapest way to get from Asansol to Goa is to take a train and bus, which takes around 2780 minutes and costs between ₹1,600 - ₹15,000 per person. 

## 3. Hotel Options
The hotel options provided are:

- The Astor - All Suites Hotel Candolim Goa: Price and other details are not provided.
- Summit Calangute Resort And Spa: Price and other details are not provided.
- The Crescent: Price and other details are not provided.
- W Goa: A 5-star hotel priced at $415 per night. Other details are not provided.
- Goa Radisson Blu: Offers amenities like private beach access. Price and other details are not provided.
- Hard Rock Hotel Goa: Known for its lively atmosphere and central location near Calangute Beach. Price and other details are not provided.

## 4. Itinerary
- Day 1: Morning exploration of the Aguada Fort, afternoon relaxation at the Palolem Beach, and an evening cruise over the serene waters. Remember to pack beach essentials.
- Day 2: Morning guided E-Bike Tour at the Old Goa Heritage & Culture, afternoon tour of Old Goa Dudhsagar Falls and Spice Plantation, and evening enjoyment of the nightlife at Aguda. Carry comfortable shoes for the bike tour.
- Day 3: Morning engagement in Nature and Wildlife Tours at the Baga Beach, afternoon visit to the Tanshikar's Working Spice Farm, and ending the day with a Dinner Cruise Party with Hotel Transfer Option. Don't forget to buy some spices from the spice farm.

## 5. Estimated Total Cost
The total cost per person for transport ranges from ₹1,600 - ₹15,000 or $301 depending on the chosen option. The hotel costs and itinerary activity costs are not provided. Therefore, the total cost cannot be estimated at this time.

In [54]:
# Test the graph for transport only
input_text= "I want to go to Goa from Asansol on 2nd October, 2025. What are the convenient transport options?"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: transport_only
{
  "origin": "Asansol",
  "city": "Goa",
  "date": "2025-10-02",
  "return_date": null,
  "days": null,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": null,
  "hotel_constraints": null
}
In transport node
Transport search query: Transport options from Asansol to Goa on 2025-10-02 for 1 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.
Raw transport search result: {'query': 'Transport options from Asansol to Goa on 2025-10-02 for 1 day(s), budget any, interests: none. Return provider, mode, depart, arrive, duration in minutes, price per person, and notes.', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.rome2rio.com/s/%C4%80sansol/Goa-Philippines', 'title': 'Āsansol to Goa - 6 ways to travel via train, plane, taxi, and bus', 'content': 'The cheapest way to get from Āsansol to Goa costs only $301, and

In [55]:
if result.get("final_plan"):
    if isinstance(result["final_plan"], dict) and "message" in result["final_plan"]:
        display(Markdown(result["final_plan"]["message"]))
    else:
        display(Markdown(result["final_plan"].content))

## Overview
The trip is planned from Asansol to Goa on the 2nd of October, 2025. The return date and duration of the stay are not specified. There are no specific budget constraints or interests mentioned for this trip.

## Transport Options
1. **Train, Plane, Taxi, Bus**: This is the quickest way to get from Asansol to Goa. The journey will take approximately 1035 minutes and the cost per person is $301.

2. **Train, Bus**: This is the cheapest way to get from Asansol to Goa. The journey will take approximately 2780 minutes and the cost per person ranges from ₹1,600 to ₹15,000.

3. **Train**: There is also an option to travel only by train from Asansol to Goa. The journey will take approximately 2675 minutes. The price for this option is not specified.

4. **Train**: For the return journey from Goa to Asansol, the train is an option. The journey will take approximately 2816 minutes. The price for this option is not specified.

5. **Train, Flight**: Another option is to travel by train and flight via Kolkata from Asansol to Goa. The journey will take approximately 600 minutes. The price for this option is not specified.

In [59]:
# Test the graph for hotel only
input_text= "I want to go to Goa on 2nd October, 2025. I want to stay for 3 nights in a 5-star hotel. provide me the best hotels"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: hotel_only
{
  "origin": null,
  "city": "Goa",
  "date": "2025-10-02",
  "return_date": null,
  "days": 3,
  "budget_per_person": null,
  "interests": null,
  "transport_constraints": null,
  "hotel_constraints": {
    "star_rating": 5
  }
}
In hotel node
Hotel search query: Hotels in Goa check-in on 2025-10-02 for 3 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.
Raw search result: {'query': 'Hotels in Goa check-in on 2025-10-02 for 3 night(s), budget any, interests: none. Return hotel name, rating, price per night, distance from center, and amenities.', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.tripadvisor.com/Hotels-g297604-Goa-Hotels.html', 'title': 'THE 10 BEST Hotels in Goa, India 2025 (from $11)', 'content': 'Seashell Suites and Villas, The Astor - All Suites Hotel Candolim Goa, and Zone Connect By The Park Calangute Goa are all popula

In [60]:
if result.get("final_plan"):
    if isinstance(result["final_plan"], dict) and "message" in result["final_plan"]:
        display(Markdown(result["final_plan"]["message"]))
    else:
        display(Markdown(result["final_plan"].content))

## Overview
The user is planning a 3-day trip to Goa, starting from October 2, 2025. The user has not specified a budget but is interested in 5-star hotel accommodations.

## Hotel Options
Here are the 5-star hotel options available for the user's stay:

1. **Seashell Suites and Villas**: This hotel does not have a specified rating or amenities. The price per night is not provided.

2. **The Astor - All Suites Hotel Candolim Goa**: This hotel does not have a specified rating or amenities. The price per night is not provided.

3. **Zone Connect By The Park Calangute Goa**: This hotel does not have a specified rating or amenities. The price per night is not provided.

4. **Goa Radisson Blu**: This hotel does not have a specified rating but offers private beach access. The price per night is not provided.

5. **Novotel Goa Resort & Spa Hotel**: This hotel does not have a specified rating or amenities. The price per night is not provided.

6. **Novotel Goa Candolim Hotel**: This hotel does not have a specified rating or amenities. The price per night is not provided.

7. **Hilton Goa Resort Candolim**: This hotel does not have a specified rating or amenities. The price per night is not provided.

8. **Taj Cidade de Goa Horizon**: This 5-star hotel offers direct beach access and luxurious amenities. The price per night is not provided.

9. **The St. Regis Goa Resort**: This 5-star hotel does not have specified amenities. The price per night is not provided.

The remaining hotels do not meet the user's 5-star rating requirement. Please note that the prices per night for all hotels are not provided.

In [39]:
# Test the graph for unrelated
input_text= "What is the capital of Maharashtra?"
result = graph.invoke({"goal": input_text})
print(result)

Supervisor raw output: unrelated
{'goal': 'What is the capital of Maharashtra?', 'intent': 'unrelated', 'final_plan': {'message': 'The request was unrelated to travel planning.'}}
